<a href="https://colab.research.google.com/github/shelinanggg/capstone-project-shelin/blob/main/capstone_project_dailyplanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-oaradtn1
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-oaradtn1
  Resolved https://github.com/ibm-granite-community/utils to commit 97732c007b2768d5c61dc61169743373043252ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [8]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)

In [9]:
# 1. Definisikan konteks dan pertanyaan untuk model
context = "Design and develop a colorful and interactive daily planner UI components."
question = "Create a daily planner. It should have a main header 'My Daily Planner'. Below it, include an input field to add new tasks and an 'Add Task' button. When the button is clicked, the new task should appear in a list below. Each task in the list should have a checkbox to mark it as complete. Use colors for the header and button to make it vibrant."
initial_tasks = ["Finish project report", "Go to the gym", "Buy groceries"]

# 2. Panggil fungsi untuk mendapatkan kode dari model
# (Langkah ini akan mengirim permintaan ke AI)
generated_code = get_answer_using_zeroshot(context, question, initial_tasks)

# 3. Tampilkan kode yang dihasilkan oleh AI
print(f"Generated Code:\n{generated_code}")

# 4. Jalankan kode yang dihasilkan secara manual atau dengan exec()
# Di bawah ini adalah contoh kode yang kemungkinan besar akan dihasilkan.

Generated Code:
```python
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display

# Create a function to add tasks
def add_task(task_text):
    task_widget = widgets.Label(value=task_text, description=True, layout=widgets.Layout(width='30%'))
    task_checkbox = widgets.Checkbox(value=False, description='Complete', disabled=False, indent=False)
    task_widget.add_class('task-item')
    task_checkbox.add_class('task-checkbox')
    task_layout = widgets.HBox([task_widget, task_checkbox])
    task_layout.add_class('task-layout')
    task_list.children = [task_layout]

# Create the main header
header = widgets.HBox([widgets.Label('My Daily Planner', layout=widgets.Layout(color='blue'))])

# Create input field and 'Add Task' button
task_input = widgets.Text(description='Enter task:', layout=widgets.Layout(width='30%'))
add_button = widgets.Button(description='Add Task', button_style='info', layout=widgets.Layout(color='green'))

# Function to handle b

In [10]:
import ipywidgets as widgets
from IPython.display import display, HTML
import datetime

# --- 1. Konfigurasi Awal dan Data ---
# Data sekarang menyertakan 'date' dan 'time'
tasks_data = [
    {'id': 1, 'text': 'Selesaikan laporan proyek', 'priority': 'high', 'completed': False, 'date': datetime.date.today(), 'time': '14:00'},
    {'id': 2, 'text': 'Pergi ke gym', 'priority': 'medium', 'completed': True, 'date': datetime.date.today() - datetime.timedelta(days=1), 'time': '18:00'},
    {'id': 3, 'text': 'Beli bahan makanan', 'priority': 'low', 'completed': False, 'date': datetime.date.today(), 'time': '10:30'},
]
next_task_id = 4

# --- 2. Fungsi Helper untuk Styling dan Logika ---

# Peta warna dan ikon untuk prioritas
priority_map = {
    'high': {'color': '#ff6b6b', 'icon': '🔥'},
    'medium': {'color': '#ffd93d', 'icon': '⚡'},
    'low': {'color': '#6bcf7f', 'icon': '🌟'}
}

def render_tasks_list():
    """Merender ulang seluruh daftar tugas berdasarkan data `tasks_data`."""
    global tasks_data

    # Logika pengurutan baru: 1. Selesai/Belum, 2. Tanggal, 3. Waktu
    tasks_data.sort(key=lambda x: (
        x['completed'],
        x.get('date', datetime.date.max),  # Urutkan berdasarkan tanggal
        x.get('time', '23:59')             # Lalu urutkan berdasarkan waktu
    ))

    task_widgets = []
    for task in tasks_data:
        task_widgets.append(create_task_widget(task))

    tasks_container.children = task_widgets

def create_task_widget(task):
    """Membuat satu widget HBox untuk sebuah tugas."""
    p_info = priority_map[task['priority']]

    # Format tanggal dan waktu untuk ditampilkan
    task_date = task.get('date', datetime.date.today()).strftime('%d %b %Y')
    task_time = task.get('time', '')

    # Teks tugas dengan efek coret dan informasi jadwal
    task_text_style = 'text-decoration: line-through; color: #888;' if task['completed'] else 'font-weight: 500;'
    task_details_html = f"""
        <div style='{task_text_style}'>{task['text']}</div>
        <div style='font-size: 0.8em; color: #555;'>🗓️ {task_date} &nbsp; ⏰ {task_time}</div>
    """
    task_details = widgets.HTML(task_details_html)

    # Tombol Aksi
    complete_button = widgets.Button(description='✅ Selesai', button_style='success', layout=widgets.Layout(width='90px'))
    delete_button = widgets.Button(description='🗑️ Hapus', button_style='danger', layout=widgets.Layout(width='90px'))

    complete_button.task_id = task['id']
    delete_button.task_id = task['id']

    complete_button.on_click(on_complete_button_clicked)
    delete_button.on_click(on_delete_button_clicked)

    action_box = widgets.HBox([complete_button, delete_button])
    if task['completed']:
        action_box.layout.display = 'none'

    task_item = widgets.HBox([
        widgets.HTML(f"<b>{p_info['icon']}</b>", layout=widgets.Layout(margin='0 10px 0 0')),
        task_details
    ], layout=widgets.Layout(flex='1'))

    return widgets.HBox(
        [task_item, action_box],
        layout=widgets.Layout(
            display='flex',
            justify_content='space-between',
            align_items='center',
            border_left=f'5px solid {p_info["color"]}',
            background_color='#f9f9f9' if not task['completed'] else '#e0e0e0',
            padding='10px',
            margin='5px 0'
        )
    )

# --- 3. Fungsi Event Handler (Aksi Pengguna) ---

def on_add_button_clicked(b):
    """Menambah tugas baru ke dalam daftar dengan tanggal dan waktu."""
    global next_task_id
    if task_input.value:
        tasks_data.append({
            'id': next_task_id,
            'text': task_input.value,
            'priority': priority_buttons.value,
            'completed': False,
            'date': date_picker.value, # Ambil nilai dari date picker
            'time': time_input.value   # Ambil nilai dari time input
        })
        next_task_id += 1
        task_input.value = ''
        render_tasks_list()

def on_complete_button_clicked(b):
    """Menandai tugas sebagai selesai."""
    task = next((t for t in tasks_data if t['id'] == b.task_id), None)
    if task:
        task['completed'] = True
        render_tasks_list()

def on_delete_button_clicked(b):
    """Menghapus tugas dari daftar."""
    global tasks_data
    tasks_data = [t for t in tasks_data if t['id'] != b.task_id]
    render_tasks_list()

# --- 4. Definisi Widget Antarmuka ---

# Atur tanggal hari ini
sekarang = datetime.datetime.now()
hari = sekarang.strftime('%A')
tanggal_lengkap = sekarang.strftime('%d %B %Y')

# Header
header = widgets.HTML(f"""
<div style="background: linear-gradient(45deg, #667eea, #764ba2); color: white; padding: 20px; text-align: center; border-radius: 15px 15px 0 0;">
    <h1 style="margin: 0; font-size: 2.5em; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">✨ My Daily Planner ✨</h1>
    <p style="margin: 5px 0 0; font-size: 1.2em;">{hari}, {tanggal_lengkap}</p>
</div>
""")

# --- Widget Baru: Date Picker dan Time Input ---
date_picker = widgets.DatePicker(
    description='Tanggal',
    value=datetime.date.today(), # Nilai default adalah hari ini
    disabled=False
)
time_input = widgets.Text(
    description='Waktu',
    value='09:00', # Nilai default
    placeholder='JJ:MM',
    layout=widgets.Layout(width='150px')
)

# Input field
task_input = widgets.Text(placeholder='Apa yang ingin kamu lakukan hari ini?', layout=widgets.Layout(width='auto', flex='1'))

# Tombol Tambah
add_button = widgets.Button(description='🚀 Tambah Task', button_style='success')
add_button.on_click(on_add_button_clicked)

# Tombol Prioritas
priority_buttons = widgets.ToggleButtons(
    options=[('🔥 Tinggi', 'high'), ('⚡ Sedang', 'medium'), ('🌟 Rendah', 'low')],
    value='high'
)

# Area Input yang digabungkan
input_area = widgets.VBox([
    widgets.HBox([task_input, add_button]),
    widgets.HBox([date_picker, time_input]),
    widgets.Label("Pilih Prioritas:"),
    priority_buttons
], layout=widgets.Layout(
    background='linear-gradient(135deg, #a8e6cf 0%, #dcedc8 100%)',
    padding='20px',
    border_radius='15px',
    margin='20px 0'
))

# Kontainer untuk daftar tugas
tasks_container = widgets.VBox([])

# --- 5. Susun Tata Letak Akhir dan Tampilkan ---

render_tasks_list()

app_layout = widgets.VBox([
    header,
    input_area,
    widgets.HTML('<h2 style="text-align: center; color: #333;">📋 Daftar Task Hari Ini</h2>'),
    tasks_container
], layout=widgets.Layout(
    width='700px', margin='0 auto', border='2px solid #ddd',
    border_radius='15px', background='rgba(255, 255, 255, 0.95)',
    box_shadow='0 10px 30px rgba(0,0,0,0.1)'
))

display(app_layout)